In [55]:
from dotenv import load_dotenv
from agents import Agent, trace, function_tool, Runner, AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import resend
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio

In [56]:
load_dotenv(override=True)

True

In [57]:
import os

from agents import OpenAIChatCompletionsModel

GEMINI_BASE_URL = os.getenv("GEMINI_BASE_URL", "https://generativelanguage.googleapis.com/v1beta/openai/")

gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=os.getenv("GEMINI_API_KEY"))

gemini_model = OpenAIChatCompletionsModel(openai_client=gemini_client, model="gemini-2.0-flash")

In [58]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [59]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model =gemini_model
    # model=LitellmModel(
    #     model="gemini/gemini-2.0-flash", api_key=os.getenv("GEMINI_API_KEY")
    # ),
)

sales_agent2 = Agent(
    name="Engaging Sales Agent", instructions=instructions2, model=gemini_model
)

sales_agent3 = Agent(
    name="Busy Sales Agent", instructions=instructions3, model=gemini_model

    )


In [60]:
sales_email_picker = Agent(
    name="sales_email_picker",
    instructions="""
You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the full selected email only. ",
    """,
    model=LitellmModel(
        model="gemini/gemini-2.0-flash", api_key=os.getenv("GEMINI_API_KEY")
    ),
)

In [61]:
import resend
resend.api_key = os.getenv("RESEND_API_KEY")
@function_tool
def send_email(subject:str, html_body:str):
    """Send out an email with the subject and html body to the user with the given body."""
    params: resend.Emails.SendParams = {
        "from": "Acme <onboarding@resend.dev>",
        "to": ["adedejiosvaldo@gmail.com"],
        "subject": subject,
        "html": html_body,
    }
    email = resend.Emails.send(params)
    print(email)

In [62]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x768679513a60>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7686a032afc0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'req

In [63]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response. Follow the rules of copywriting \""

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
formatting (like **bold** or *italic* or [links](https://example.com)) \
Be sure to give spacing between paragraphs, \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(
    name="Email subject writer", instructions=subject_instructions, model=gemini_model
)
subject_tool = subject_writer.as_tool(
    tool_name="subject_writer",
    tool_description="Write a subject for a cold sales email",
)

html_converter = Agent(
    name="HTML email body converter", instructions=html_instructions, model=gemini_model
)
html_tool = html_converter.as_tool(
    tool_name="html_converter",
    tool_description="Convert a text email body to an HTML email body",
)

In [64]:
email_tools = [subject_tool, html_tool, send_email]

In [65]:
instructions = "You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, \
then use the html_converter tool to convert the body to HTML. \
Fill in the information in the email to make it more personalized and relevant. \
Finally, you use the send_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model=gemini_model,
    handoff_description="Convert an email to HTML and send it",
)

In [66]:
handoffs=[emailer_agent]

In [67]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective at converting. \
After picking the email, you handoff to the Email Manager agent to format and send the email. \
Your name is Alice, and you are sending the email to the CEO of a company."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model=gemini_model
)
message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

{'id': '2dde2fc1-f0cf-4346-8302-88ea1d71f38f'}


In [68]:
#Guidrails
from pydantic import BaseModel
class NameCheckOutput(BaseModel):
    name: str
    is_name_in_message: bool

guiderail_agent=Agent(
    name="Name Checker",
    instructions="Check if the user is including someone else's name in the message.",
    model=gemini_model,
   output_type=NameCheckOutput
)

In [70]:
from agents import GuardrailFunctionOutput, input_guardrail


@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guiderail_agent, message,context=ctx)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={
        "is_name_in_message": is_name_in_message
    },tripwire_triggered=is_name_in_message)


In [ ]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective at converting. \
After picking the email, you handoff to the Email Manager agent to format and send the email. "


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model=gemini_model,
    input_guardrails=[guardrail_against_name]
)
message = "Send out a cold sales email addressed to Dear CEO"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)